In [9]:
import pandas as pd
import folium
from folium.plugins import MiniMap
from pathlib import Path

# === 1. Baca data dari Google Sheets ===
url = "https://docs.google.com/spreadsheets/d/1iNevuUEhHf16MPlA-WDD1vDxiTRc5A4hGeBHZjh4XPk/export?format=csv"
data = pd.read_csv(url)

# Cek kolom
print(data.head())

# Rapikan nama kolom
data.columns = data.columns.str.strip().str.upper()

# === 2. Tentukan lokasi awal peta ===
center_lat = data["LINTANG"].mean()
center_lon = data["BUJUR"].mean()

# === 3. Buat Peta Utama ===
m = folium.Map(location=[center_lat, center_lon], zoom_start=6, tiles="OpenStreetMap")

# Tambah basemap lain
folium.TileLayer("cartodb positron", name="Carto Light").add_to(m)
folium.TileLayer("cartodb dark_matter", name="Carto Dark").add_to(m)
folium.TileLayer(
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr="Tiles © Esri",
    name="Esri Satellite",
    overlay=False,
    control=True
).add_to(m)

# MiniMap
MiniMap(toggle_display=True, position="bottomright").add_to(m)

# === 4. Tambahkan titik berdasarkan JENIS ===
for jenis in data["JENIS"].unique():
    group = folium.FeatureGroup(name=f"{jenis}", show=True).add_to(m)
    subset = data[data["JENIS"] == jenis]

    for _, row in subset.iterrows():
        popup_text = f"""
        <b>KELOMPOK:</b> {row['KELOMPOK']}<br>
        <b>SPESIES:</b> {row['SPESIES']}<br>
        <b>NAMA_ID:</b> {row['NAMA_ID']}<br>
        <b>JUMLAH TEMU:</b> {row['JUMLAH_TEMU']}<br>
        <b>DESKRIPSI:</b> {row['DESKRIPSI']}<br>
        <b>TAHUN:</b> {row['TAHUN']}<br>
        <b>LINK LAPOR:</b> <a href="{row['LINK_LAPOR']}" target="_blank">Klik di sini</a>
        """

        folium.Marker(
            location=[row["LINTANG"], row["BUJUR"]],
            popup=folium.Popup(popup_text, max_width=500),
            tooltip=row["SPESIES"],
            icon=folium.Icon(color="green", icon="info-sign")
        ).add_to(group)

# === 5. Layer Control ===
folium.LayerControl(collapsed=False).add_to(m)

# === 6. Tambahkan Judul Sticky dengan Keterangan dan Link ===
title_html = '''
     <div style="
         position: fixed;
         top: 15px;
         left: 50%;
         transform: translateX(-50%);
         background-color: rgba(255, 255, 255, 0.85);
         padding: 15px 30px;
         border-radius: 14px;
         font-family: 'Segoe UI', Arial, sans-serif;
         text-align: center;
         box-shadow: 0 3px 8px rgba(0,0,0,0.25);
         z-index: 9999;
         max-width: 95%;
         backdrop-filter: blur(6px);
     ">
         <div style="font-size: 22px; font-weight: 700; color: #1f3b73;">
             PETA SEBARAN BIOTA LAUT DILINDUNGI
         </div>
         <div style="font-size: 16px; margin-top: 6px; line-height: 1.5; color: #2c3e50;">
             Buka Grafik Monitoring : 
             <a href="https://lookerstudio.google.com/u/0/reporting/faefd501-627c-47bf-b741-b25ebb73af8c/page/p_tu61f29uwd" 
                target="_blank" style="color: #007BFF; text-decoration: none; font-weight: 600;">
                Klik Disini
             </a>
             &nbsp;|&nbsp;
             Buka Dashboard : 
             <a href="https://sites.google.com/view/biota-laut-dilindungi/home" 
                target="_blank" style="color: #007BFF; text-decoration: none; font-weight: 600;">
                SI MOLI
             </a>
         </div>
     </div>
'''
m.get_root().html.add_child(folium.Element(title_html))

# === 6. Simpan ke HTML ===
html_str = m.get_root().render()
Path("index.html").write_text(html_str, encoding="utf-8")

print("✅ Peta berhasil dibuat → buka file: index.html")


              PROVINSI KABUPATEN KECAMATAN  WPP DESA  HARI  BULAN  TAHUN  \
0  Nusa Tenggara Timur    Kupang       NaN  573  NaN    21      4   2020   
1  Nusa Tenggara Timur    Kupang       NaN  573  NaN    22      4   2020   
2  Nusa Tenggara Timur    Kupang       NaN  573  NaN    22      4   2020   
3  Nusa Tenggara Timur    Kupang       NaN  573  NaN     7      8   2020   
4  Nusa Tenggara Timur    Kupang       NaN  573  NaN    21      4   2020   

    LINTANG       BUJUR  ... KONDISI AKDATA_M JUMLAH_TEMU DALAM_M ELEVASI_M  \
0 -9.298102  123.965107  ...       1        3         3.0     0.0         0   
1 -9.350743  123.951744  ...       1        3       124.0     0.0         0   
2 -9.407844  123.873807  ...       1        3        12.0     0.0         0   
3 -9.412483  123.875333  ...       1        3         5.0     0.0         0   
4 -9.421808  123.857970  ...       1        3        13.0     0.0         0   

      OBSERVASI   INSTITUSI       LISENSI  \
0  Diatas kapal  Pemeri